#### 상대 모멘텀
- 10개의 종목을 이용하여 투자 전략 시스템
    1. 월초부터 월말까지의 수정종가를 이용하여 수익율 생성
    2. 월별 수익율이 높은 n개
    3. 해당하는 종목을 매수하여 해당하는 종목에 신호가 없어지면 매도하는 형식으로 수익률 생성

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import warnings

In [2]:
# warning 메시지 출력 방지
warnings.filterwarnings('ignore')

In [3]:
# 월별 수익율 계산하는 함수
def create_1m_rtn(_df,
                  _ticker, 
                  _start='2010-01-01',
                  _col = 'Adj Close'
                  ):
    
    # 복사본 생성
    result = _df.copy()
    
    # Date 컬럼이 존재하면 index로 변경
    if 'Date' in result.columns:
        result.set_index('Date',inplace=True)
    # 인덱스를 시계열로 변경
    result.index = pd.to_datetime(result.index)
    
    # 시작 시간과 기준이 되는 컬럼 필터링
    result = result.loc[_start:,[_col]]
    
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    
    # 월별 수익률 생성
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    
    # 기준 년월의 유니크한 리스트 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [4]:
aapl = pd.read_csv('../csv/AAPL.csv')

In [5]:
sample_aapl, ym_list = create_1m_rtn(aapl, 'AAPL')

In [6]:
sample_aapl

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
...,...,...,...,...
2019-06-18,198.449997,2019-06,0,AAPL
2019-06-19,197.869995,2019-06,0,AAPL
2019-06-20,199.460007,2019-06,0,AAPL


In [7]:
ym_list

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', 

In [8]:
# listdir 은 파일의 목록을 만들어 줌
os.listdir('./data')

['AAPL.csv',
 'AMZN.csv',
 'BND.csv',
 'GDX.csv',
 'GLD.csv',
 'GM.csv',
 'MSFT.csv',
 'SLV.csv',
 'SPY.csv',
 'USM.csv']

In [9]:
# glob는 파일의 경로와 이름의 목록을 나열해줌
files = glob('./data/*.csv')

In [10]:
files

['./data\\AAPL.csv',
 './data\\AMZN.csv',
 './data\\BND.csv',
 './data\\GDX.csv',
 './data\\GLD.csv',
 './data\\GM.csv',
 './data\\MSFT.csv',
 './data\\SLV.csv',
 './data\\SPY.csv',
 './data\\USM.csv']

In [11]:
# 반복문을 이용

# 종목별 데이터프레임 생성

# 월말의 데이터들을 모아주는 작업
# 월별 수익률 같이 계산
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

for file in files:
    # 경로와 파일명을 나눠준다.
    folder, name = os.path.split(file)  
    # tail : AAPL.csv --> AAPL    .csv   로 나눌거임
    head, tail = os.path.splitext(name)
    # head 변수는 create_1m_rtn 함수에서 _ticker에 사용
    
    # csv 파일 로드
    read_df = pd.read_csv(file)
    
    #create_1m_rtn() 함수 사용
    price_df, ym_list = create_1m_rtn(read_df,_ticker = head)
    
    # price_df 를 stock_df에 단순 행결합 모두 결합
    stock_df = pd.concat([stock_df,price_df])
    
    # 두번째 반복문 생성
    # price_df의 월별 수익율 계산하는 반복문
    for ym in ym_list:
        # ym : 기준 년월 
        flag = price_df['STD-YM'] == ym
        # 월초 수정 종가
        buy  = price_df.loc[flag].iloc[0,0]
        # 월말 수정 종가
        sell = price_df.loc[flag].iloc[-1,0]
        # 수익률 계산
        m_rtn = sell / buy
        # price_df에 flag 조건과 1m rtn 컬럼에 대입
        price_df.loc[flag, '1m_rtn'] = m_rtn
        
        
        # 월말의 데이터를 month_last_df에 단순 행결합
        last_data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat([month_last_df,last_data])

In [12]:
# 데이터 백업
month_rtn_df = month_last_df.copy()

In [13]:
month_rtn_df

,CODE,1m_rtn
Date,,
2010-01-29,AAPL,0.897435
2010-02-26,AAPL,1.050789
2010-03-31,AAPL,1.124456
2010-04-30,AAPL,1.106454
2010-05-28,AAPL,0.964445
...,...,...
2019-02-28,USM,0.817688
2019-03-29,USM,0.974942
2019-04-30,USM,1.035968


In [14]:
# 인덱스 리셋
month_rtn_df.reset_index(inplace = True)
month_rtn_df

,Date,CODE,1m_rtn
0,2010-01-29,AAPL,0.897435
1,2010-02-26,AAPL,1.050789
2,2010-03-31,AAPL,1.124456
3,2010-04-30,AAPL,1.106454
4,2010-05-28,AAPL,0.964445
...,...,...,...
1125,2019-02-28,USM,0.817688
1126,2019-03-29,USM,0.974942
1127,2019-04-30,USM,1.035968
1128,2019-05-31,USM,0.894067


In [15]:
# pivot_table을 이용하여 테이블을 재구조화
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date',
    columns = 'CODE',
    values = '1m_rtn'
)
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.897435,0.936595,1.011692,0.853490,0.965027,NaN,0.910501,0.922809,0.947586,0.870922
2010-02-26,1.050789,0.996046,1.003909,1.022124,1.009968,NaN,1.013871,0.982274,1.015404,1.004938
2010-03-31,1.124456,1.090172,0.996728,0.994848,0.995614,NaN,1.009304,1.063935,1.049976,1.117472
2010-04-30,1.106454,1.040134,1.011023,1.088577,1.046254,NaN,1.047325,1.041049,1.008574,0.996684
2010-05-28,0.964445,0.912503,1.011821,0.999599,1.027218,NaN,0.839806,0.978827,0.908766,0.968954
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,1.044274,1.008363,1.001883,0.985379,0.995904,1.018051,1.094653,0.980563,1.031919,0.817688
2019-03-29,1.085615,1.065214,1.022547,1.033180,1.001067,0.947842,1.048076,0.998592,1.011783,0.974942
2019-04-30,1.049310,1.061917,1.003719,0.951275,0.997285,1.031515,1.097295,0.990813,1.028653,1.035968


In [16]:
# 행 단위로 수익률 순위를 백분율로 계산
month_rtn_df = month_rtn_df.rank(
    axis=1,
    ascending=False,
    method = 'max',
    pct = True
)

In [17]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.777778,0.444444,0.111111,1.000000,0.222222,NaN,0.666667,0.555556,0.333333,0.888889
2010-02-26,0.111111,0.888889,0.777778,0.222222,0.555556,NaN,0.444444,1.000000,0.333333,0.666667
2010-03-31,0.111111,0.333333,0.777778,1.000000,0.888889,NaN,0.666667,0.444444,0.555556,0.222222
2010-04-30,0.111111,0.666667,0.777778,0.222222,0.444444,NaN,0.333333,0.555556,0.888889,1.000000
2010-05-28,0.666667,0.777778,0.222222,0.333333,0.111111,NaN,1.000000,0.444444,0.888889,0.555556
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,0.200000,0.500000,0.600000,0.800000,0.700000,0.4,0.100000,0.900000,0.300000,1.000000
2019-03-29,0.100000,0.200000,0.500000,0.400000,0.700000,1.0,0.300000,0.800000,0.600000,0.900000
2019-04-30,0.300000,0.200000,0.700000,1.000000,0.800000,0.5,0.100000,0.900000,0.600000,0.400000


In [18]:
# 상위 15% 종목만 선택
# where(조건식, 조건식이 거짓인 경우 변경할 데이터 값)
month_rtn_df = month_rtn_df.where(month_rtn_df < 0.40,0)

In [19]:
# 0이 아닌 데이터는 1로 변환
month_rtn_df[month_rtn_df != 0 ] = 1

In [20]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2010-02-26,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2010-03-31,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2010-04-30,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2010-05-28,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2019-03-29,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2019-04-30,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
# CODE 값들로 이루어진 리스트 생성
stock_codes = stock_df['CODE'].unique()
# month_rtn_df.columns

In [22]:
month_rtn_df.loc['2010-01-29',month_rtn_df.loc['2010-01-29'] == 1 ]   

CODE
BND    1.0
GLD    1.0
SPY    1.0
Name: 2010-01-29 00:00:00, dtype: float64

In [23]:
sig_dict = dict()

for date in month_rtn_df.index:
    
    # 값이 1인 종목만 골라내서 index (종목명) 리스트로 변환
    ticker_list = list(month_rtn_df.loc[date][month_rtn_df.loc[date] == 1].index)

    # 날짜별 시그널 저장
    sig_dict[date] = ticker_list


In [24]:
sig_dict

{Timestamp('2010-01-29 00:00:00'): ['BND', 'GLD', 'SPY'],
 Timestamp('2010-02-26 00:00:00'): ['AAPL', 'GDX', 'SPY'],
 Timestamp('2010-03-31 00:00:00'): ['AAPL', 'AMZN', 'USM'],
 Timestamp('2010-04-30 00:00:00'): ['AAPL', 'GDX', 'MSFT'],
 Timestamp('2010-05-28 00:00:00'): ['BND', 'GDX', 'GLD'],
 Timestamp('2010-06-30 00:00:00'): ['BND', 'GDX', 'GLD'],
 Timestamp('2010-07-30 00:00:00'): ['MSFT', 'SPY', 'USM'],
 Timestamp('2010-08-31 00:00:00'): ['GDX', 'GLD', 'SLV'],
 Timestamp('2010-09-30 00:00:00'): ['AAPL', 'AMZN', 'SLV'],
 Timestamp('2010-10-29 00:00:00'): ['AMZN', 'MSFT', 'SLV'],
 Timestamp('2010-11-30 00:00:00'): ['AMZN', 'GDX', 'SLV'],
 Timestamp('2010-12-31 00:00:00'): ['MSFT', 'SLV', 'USM'],
 Timestamp('2011-01-31 00:00:00'): ['AAPL', 'BND', 'SPY'],
 Timestamp('2011-02-28 00:00:00'): ['GDX', 'GLD', 'SLV'],
 Timestamp('2011-03-31 00:00:00'): ['AMZN', 'SLV', 'USM'],
 Timestamp('2011-04-29 00:00:00'): ['AMZN', 'GLD', 'SLV'],
 Timestamp('2011-05-31 00:00:00'): ['AAPL', 'BND', 'USM']

In [25]:
stock_df.head(2)

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL


In [26]:
# 거래내역 컬럼을 생성하는 함수
def create_trade_book(_df,_code):
    # 복사본 생성
    book = _df[_code].copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _code:
        book[f'p_{c}'] = ''
        book[f'r_{c}'] = ''
    return book

In [27]:
# stock_df를 재구조화
stock_c_matrix = stock_df.reset_index().pivot_table(
    index='Date',
    columns ='CODE',
    values = stock_df.columns[0]
)
# 종목별 수정종가
stock_c_matrix

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.230000,93.675278,36.015179
2010-01-05,26.829010,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.510000,93.923241,35.998024
2010-01-06,26.402260,132.250000,60.766037,46.443077,111.510002,NaN,24.382378,17.860001,93.989357,35.680672
2010-01-07,26.353460,130.000000,60.719822,46.217175,110.820000,NaN,24.128809,17.889999,94.386139,35.208931
2010-01-08,26.528664,133.520004,60.781410,46.913723,111.370003,NaN,24.295214,18.150000,94.700218,34.651424
...,...,...,...,...,...,...,...,...,...,...
2019-06-18,198.449997,1901.369995,82.397118,23.670000,127.120003,36.700001,135.160004,14.050000,290.984741,50.540001
2019-06-19,197.869995,1908.790039,82.676468,24.000000,127.889999,36.779999,135.690002,14.170000,291.641541,50.040001
2019-06-20,199.460007,1918.189941,82.806168,25.049999,131.110001,36.959999,136.949997,14.450000,294.427979,49.320000


In [28]:
book = create_trade_book(stock_c_matrix,stock_codes)
book.columns

Index(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY', 'USM',
       'STD-YM', 'p_AAPL', 'r_AAPL', 'p_AMZN', 'r_AMZN', 'p_BND', 'r_BND',
       'p_GDX', 'r_GDX', 'p_GLD', 'r_GLD', 'p_GM', 'r_GM', 'p_MSFT', 'r_MSFT',
       'p_SLV', 'r_SLV', 'p_SPY', 'r_SPY', 'p_USM', 'r_USM'],
      dtype='object', name='CODE')

In [29]:
sig_dict.items()

dict_items([(Timestamp('2010-01-29 00:00:00'), ['BND', 'GLD', 'SPY']), (Timestamp('2010-02-26 00:00:00'), ['AAPL', 'GDX', 'SPY']), (Timestamp('2010-03-31 00:00:00'), ['AAPL', 'AMZN', 'USM']), (Timestamp('2010-04-30 00:00:00'), ['AAPL', 'GDX', 'MSFT']), (Timestamp('2010-05-28 00:00:00'), ['BND', 'GDX', 'GLD']), (Timestamp('2010-06-30 00:00:00'), ['BND', 'GDX', 'GLD']), (Timestamp('2010-07-30 00:00:00'), ['MSFT', 'SPY', 'USM']), (Timestamp('2010-08-31 00:00:00'), ['GDX', 'GLD', 'SLV']), (Timestamp('2010-09-30 00:00:00'), ['AAPL', 'AMZN', 'SLV']), (Timestamp('2010-10-29 00:00:00'), ['AMZN', 'MSFT', 'SLV']), (Timestamp('2010-11-30 00:00:00'), ['AMZN', 'GDX', 'SLV']), (Timestamp('2010-12-31 00:00:00'), ['MSFT', 'SLV', 'USM']), (Timestamp('2011-01-31 00:00:00'), ['AAPL', 'BND', 'SPY']), (Timestamp('2011-02-28 00:00:00'), ['GDX', 'GLD', 'SLV']), (Timestamp('2011-03-31 00:00:00'), ['AMZN', 'SLV', 'USM']), (Timestamp('2011-04-29 00:00:00'), ['AMZN', 'GLD', 'SLV']), (Timestamp('2011-05-31 00:00:

In [30]:
for date,values in sig_dict.items():
    for stock in values:
        book.loc[date,f'p_{stock}'] = f'ready_{stock}'

In [31]:
book[['p_AMZN']].value_counts()

p_AMZN    
              2337
ready_AMZN      47
Name: count, dtype: int64

In [32]:
# ready 확인하고 buy 생성

def trading(_df,s_codes):
    std_ym = ''
    buy_phase = False
    _book = _df.copy()
    #종목 별로 순회 (반복)
    for code in s_codes:
        #인덱스별 반복
        for idx in _book.index:
            # 특정 종목 코드에서 포지션을 잡는다
            if (_book.loc[idx,f'p_{code}']=='') & (_book.shift().loc[idx,f'p_{code}'] == f'ready_{code}'):
                std_ym = _book.loc[idx,'STD-YM']
                buy_phase = True
            # 해당 종목 코드에서 신호가 잡혀있다면 매수 유지
            if (_book.loc[idx,f'p_{code}']=='') & (_book.loc[idx, 'STD-YM'] == std_ym) & (buy_phase):
                _book.loc[idx,f'p_{code}']= f'buy_{code}'
            if (_book.loc[idx, f'p_{code}']==''):
                std_ym = ''
                buy_phase = False
    return _book

In [33]:
book2 = trading(book, stock_codes)

In [34]:
book2['p_MSFT'].value_counts()

p_MSFT
              1472
buy_MSFT       869
ready_MSFT      43
Name: count, dtype: int64

In [35]:
book2.head(2)

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM,...,p_GM,r_GM,p_MSFT,r_MSFT,p_SLV,r_SLV,p_SPY,r_SPY,p_USM,r_USM
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,26.782711,133.899994,60.611969,44.908779,109.800003,NaN,24.525019,17.23,93.675278,36.015179,...,,,,,,,,,,
2010-01-05,26.829010,134.690002,60.789135,45.341774,109.699997,NaN,24.532942,17.51,93.923241,35.998024,...,,,,,,,,,,


In [36]:
# 왜 안되는지 모르는 내 코드

# 수익율 계산 함수
def multi_return(_df, s_codes):
    #복사본 생성
    _book = _df.copy()
    rtn = 1
    buy_dict = dict()
    sell_dict = dict()
    
    for idx in _book.index:
        for code in s_codes:
            # 매수 타이밍 -> 2일 전에 '' 1일 전에 'ready' 오늘이 'buy'인 타이밍
            if (_book.shift(2).loc[idx,f'p_{code}']=='') & (_book.shift(1).loc[idx,f'p_{code}']==f'ready_{code}') & (_book.loc[idx,f'p_{code}']==f'buy_{code}'):
                # 해당하는 날짜에 특정 종목의 수정 종가
                buy_dict[code] = _book.loc[idx,code]
                print(f' 매수일 : {idx}, 매수 가격 : {buy_dict[code]}, 종목코드 :{code}')
            # 매도 타이밍 -> 1일 전에 'buy' 오늘이 ''인 상태
            elif (_book.shift().loc[idx,f'p_{code}']==f'buy_{code}') & (_book.loc[idx,f'p_{code}']==''):
                sell_dict[code] = _book.loc[idx,code]
                
                # 수익률 계산
                rtn = sell_dict[code]/buy_dict[code]
                _book.loc[idx, f'r_{code}'] = rtn
                print(f' 매도일 : {idx}, 매도 가격 : {buy_dict[code]}, 종목코드 :{code}, 수익율 : {rtn}')
                
                #buy_dict, sell_dict 초기화
            if _book.loc[idx, f'p_{code}'] == '':
                buy_dict[code] = 0
                sell_dict[code] = 0
    return _book

In [37]:
# # 왜 되는 종호형 코드
# def multi_return(_df, s_codes):
#     _book = _df.copy()
#     rtn = 1
#     buy_dict = dict()
#     sell_dict = dict()

#     for idx in _book.index:
#         for code in s_codes:
#             if (_book.shift(2).loc[idx, f'p_{code}'] == '') & \
#                 (_book.shift(1).loc[idx, f'p_{code}'] == f'ready_{code}') & \
#                 (_book.loc[idx, f'p_{code}'] == f'buy_{code}'):
#                 buy_dict[code] = _book.loc[idx, code]
#                 print(f'매수일 : {idx}, 매수가 : {buy_dict[code]}, 종목코드 : {code}')
#             elif (_book.shift(1).loc[idx, f'p_{code}'] == f'buy_{code}') & \
#                 (_book.loc[idx, f'p_{code}'] == ''):
#                 sell_dict[code] = _book.loc[idx, code]
#                 rtn = sell_dict[code] / buy_dict[code]
#                 _book.loc[idx, f'r_{code}'] == rtn
#                 print(f'매도일 : {idx}, 매도가 : {sell_dict[code]}, 종목코드 : {code}, 수익률 : {rtn}')
#             if _book.loc[idx, f'p_{code}'] == '':
#                 buy_dict[code] = 0
#                 sell_dict[code] = 0
#     return _book

In [38]:
rtn_book = multi_return(book2, stock_codes)

 매수일 : 2010-02-01 00:00:00, 매수 가격 : 61.280487, 종목코드 :BND
 매수일 : 2010-02-01 00:00:00, 매수 가격 : 108.349998, 종목코드 :GLD
 매수일 : 2010-02-01 00:00:00, 매수 가격 : 90.145805, 종목코드 :SPY
 매수일 : 2010-03-01 00:00:00, 매수 가격 : 26.154476, 종목코드 :AAPL
 매도일 : 2010-03-01 00:00:00, 매도 가격 : 61.280487, 종목코드 :BND, 수익율 : 1.0049718273289832
 매수일 : 2010-03-01 00:00:00, 매수 가격 : 42.019035, 종목코드 :GDX
 매도일 : 2010-03-01 00:00:00, 매도 가격 : 108.349998, 종목코드 :GLD, 수익율 : 1.0099677159200318
 매수일 : 2010-04-01 00:00:00, 매수 가격 : 131.809998, 종목코드 :AMZN
 매도일 : 2010-04-01 00:00:00, 매도 가격 : 42.019035, 종목코드 :GDX, 수익율 : 1.0394266550862008
 매도일 : 2010-04-01 00:00:00, 매도 가격 : 90.145805, 종목코드 :SPY, 수익율 : 1.0845873083056943
 매수일 : 2010-04-01 00:00:00, 매수 가격 : 36.212452, 종목코드 :USM
 매도일 : 2010-05-03 00:00:00, 매도 가격 : 131.809998, 종목코드 :AMZN, 수익율 : 1.043092383629351
 매수일 : 2010-05-03 00:00:00, 매수 가격 : 46.951374, 종목코드 :GDX
 매수일 : 2010-05-03 00:00:00, 매수 가격 : 24.56805, 종목코드 :MSFT
 매도일 : 2010-05-03 00:00:00, 매도 가격 : 36.212452, 종목코드 :USM, 수익율 : 1.

In [39]:
rtn_book.isna().sum()

CODE
AAPL        0
AMZN        0
BND         0
GDX         0
GLD         0
GM        222
MSFT        0
SLV         0
SPY         0
USM         0
STD-YM      0
p_AAPL      0
r_AAPL      0
p_AMZN      0
r_AMZN      0
p_BND       0
r_BND       0
p_GDX       0
r_GDX       0
p_GLD       0
r_GLD       0
p_GM        0
r_GM        0
p_MSFT      0
r_MSFT      0
p_SLV       0
r_SLV       0
p_SPY       0
r_SPY       0
p_USM       0
r_USM       0
dtype: int64

In [40]:
def multi_acc_return(_df, s_codes):
    _book = _df.copy()
    acc_rtn = 1
    
    for idx in _book.index:
        count = 0
        rtn = 0
        for code in s_codes:
            # 수익률이 존재하는가?
            if _book.loc[idx,f'r_{code}']:
                count += 1
                rtn += _book.loc[idx, f'r_{code}']
        if(rtn != 0) & (count != 0):
            acc_rtn *= (rtn /count)
            print(f'누적 - 매도일 : {idx}, 매도 종목수 : {count}, 수익율 : {round(rtn/count,2)}') # round 함수 -> 반올림 함수
        _book.loc[idx, 'acc_rtn'] = acc_rtn
    print(f'총 누적 수익율은 {acc_rtn}입니다.')
    return _book, acc_rtn

In [41]:
multi_acc_return(rtn_book, stock_codes)

누적 - 매도일 : 2010-03-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.01
누적 - 매도일 : 2010-04-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.06
누적 - 매도일 : 2010-05-03 00:00:00, 매도 종목수 : 2, 수익율 : 1.03
누적 - 매도일 : 2010-06-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.05
누적 - 매도일 : 2010-08-02 00:00:00, 매도 종목수 : 3, 수익율 : 0.98
누적 - 매도일 : 2010-09-01 00:00:00, 매도 종목수 : 3, 수익율 : 0.93
누적 - 매도일 : 2010-10-01 00:00:00, 매도 종목수 : 2, 수익율 : 1.07
누적 - 매도일 : 2010-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.08
누적 - 매도일 : 2010-12-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.97
누적 - 매도일 : 2011-01-03 00:00:00, 매도 종목수 : 2, 수익율 : 1.11
누적 - 매도일 : 2011-02-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.15
누적 - 매도일 : 2011-03-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.01
누적 - 매도일 : 2011-04-01 00:00:00, 매도 종목수 : 2, 수익율 : 0.99
누적 - 매도일 : 2011-05-02 00:00:00, 매도 종목수 : 1, 수익율 : 0.94
누적 - 매도일 : 2011-06-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.04
누적 - 매도일 : 2011-07-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.0
누적 - 매도일 : 2011-08-01 00:00:00, 매도 종목수 : 3, 수익율 : 1.02
누적 - 매도일 : 2011-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.96
누적 - 매도일 : 

(CODE              AAPL         AMZN        BND        GDX         GLD  \
 Date                                                                    
 2010-01-04   26.782711   133.899994  60.611969  44.908779  109.800003   
 2010-01-05   26.829010   134.690002  60.789135  45.341774  109.699997   
 2010-01-06   26.402260   132.250000  60.766037  46.443077  111.510002   
 2010-01-07   26.353460   130.000000  60.719822  46.217175  110.820000   
 2010-01-08   26.528664   133.520004  60.781410  46.913723  111.370003   
 ...                ...          ...        ...        ...         ...   
 2019-06-18  198.449997  1901.369995  82.397118  23.670000  127.120003   
 2019-06-19  197.869995  1908.790039  82.676468  24.000000  127.889999   
 2019-06-20  199.460007  1918.189941  82.806168  25.049999  131.110001   
 2019-06-21  198.779999  1911.300049  82.576698  25.209999  131.979996   
 2019-06-24  199.169998  1907.953857  82.726349  25.703501  133.501907   
 
 CODE               GM        MSFT  